In [2]:
!pip install spotpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 62.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
astropy 6.0.1 requires numpy<2,>=1.22, but you have numpy 2.2.6 which is incompatible.
pandas 2.2.1 requires numpy<2,>=1.23.2; python_version == "3.11", but you have numpy 2.2.6 which is incompatible.
scikit-learn 1.4.1.post1 requires numpy<2.0,>=1.19.5, but you have numpy 2.2.6 which is incompatible.
timezonefinder 6.4.1 requires numpy<

In [3]:
# imports
from nextgen_calibration_utils import process_usgs_streamflow, run_spotpy

In [4]:
# Define inputs for calibration

gage_id = "10109001"
feature_id = 2861391
start_date = "2021-10-01"
end_date = "2022-09-30"
training_start_date = "2021-11-01"

observed_flow_path = f"/shared/calibration/{gage_id}_observed_flow.pkl"
troute_output_path = f"/home/jovyan/ngiab_preprocess_output/gage-{gage_id}/outputs/troute/troute_output_202110010000.nc"
cfe_dir = f"/home/jovyan/ngiab_preprocess_output/gage-{gage_id}/config/cat_config/CFE"
data_dir = f"/home/jovyan/ngiab_preprocess_output/gage-{gage_id}"

# Optional: Retrieve and save observed flow
# process_usgs_streamflow(gage_id, start_date, end_date, output_path=observed_flow_path)

best_params = run_spotpy(gage_id, start_date, end_date, training_start_date,
                         observed_flow_path, troute_output_path, cfe_dir,
                         data_dir, feature_id, repetitions=3)


Initializing the  Shuffled Complex Evolution (SCE-UA) algorithm  with  3  repetitions
The objective function will be minimized
Starting burn-in sampling...
Directory '/home/jovyan/ngiab_preprocess_output/gage-10109001/config/cat_config/CFE' does not exist.
WARN: pydantic version: Required(v1), Found(2.7.4).
WARN: numpy version: Required(1.26.4), Found(2.2.6).
Required dependencies not found in system path, looking into /ngen/.venv/
Valid dependencies found in venv: /ngen/.venv/
*****************
Error: Directory /home/jovyan/ngiab_preprocess_output/gage-10109001/forcings does not exist.


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/ngiab_preprocess_output/gage-10109001/outputs/troute/troute_output_202110010000.nc'

In [ ]:
!pwd